In [1]:
versioninfo()

Julia Version 1.0.0
Commit 5d4eaca0c9 (2018-08-08 20:58 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Pentium(R) CPU B980 @ 2.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.0 (ORCJIT, sandybridge)


## Common packages

In [2]:
using Printf
using LinearAlgebra
using Random
using FFTW

## External libraries

In [3]:
include("../extlibs/extlibs.jl");

## Constants

In [4]:
const Ry2eV = 13.605693009;
const ANG2BOHR = 1.8897261254578281;

## Type aliases

In [5]:
const BlochWavefunc = Array{Array{ComplexF64,2},1};
const Wavefunc = Array{ComplexF64,2};

## Atoms, lattice, etc

In [6]:
include("../src/Atoms.jl");

include("../src/gen_lattice_pwscf.jl");

include("../src/spglib.jl");

include("../src/KPoints.jl");

include("../src/XSF_utils.jl");

include("../src/Utilities.jl");

include("../src/ortho_gram_schmidt.jl");
include("../src/ortho_sqrt.jl");
include("../src/ortho_check.jl");

include("../src/Ylm_real.jl");
include("../src/Ylm_complex.jl");

include("../src/fft_support.jl");

include("../src/PWGrid.jl");

op_nabla_dot (generic function with 1 method)

In [11]:
include("PsPot_GTH_mod.jl");

eval_proj_G (generic function with 2 methods)

In [12]:
include("../src/wrappers_fft.jl");
include("../src/Electrons.jl");
include("../src/calc_strfact.jl");
include("../src/init_V_coulomb_G.jl");

mutable struct XCFuncType
end
include("../src/LDA_VWN.jl");
include("../src/GGA_PBE.jl");

## Gaussian auxiliary charge density

By convention this auxiliary charge density is negative

$$
\rho_{\mathrm{aux}}(\mathbf{r}) =
-\frac{(2\eta)^3}{(2\pi)^{3/2}}
\sum_{I=1}^{P} Z_{I}
\exp\left[-2\eta^2(\mathbf{r} - \mathbf{R}_{I})^2\right]
$$

The multiplying factor is to ensure normalization of charge density.

In [7]:
function gen_Rhoe_aux(
    eta::Float64, atoms::Atoms, Zvals::Array{Float64,1}, pw::PWGrid
)
    #
    Npoints = prod(pw.Ns)
    Rhoe_aux = zeros(Float64,Npoints)
    Natoms = atoms.Natoms
    atm2species = atoms.atm2species
    #
    for ip = 1:Npoints
        r = pw.r[:,ip]
        for ia = 1:Natoms
            isp = atm2species[ia]
            R = atoms.positions[:,ia]
            Z = Zvals[isp]
            dr2 = dot(r-R,r-R)
            Rhoe_aux[ip] = Rhoe_aux[ip] + Z*exp(-2*eta^2*dr2)
        end
    end
    return -(2*eta)^3/((2*pi)^1.5)*Rhoe_aux
end

gen_Rhoe_aux (generic function with 1 method)